### 编辑距离：从一个字符串变到另一个字符串的操作，插入，替换，删除

+ DNA检测
+ 判重、查重（两篇文章之间的重复度）
+ 搜狗拼音（对于打入的拼音可以给出推测可能的结果）
+ 机器翻译与原文对比相似度

In [1]:
from functools import lru_cache

#### 相当于每次都会有三个分叉（增加，删除，替换），把所有可能都找出来，得到步数最小的那个，所以solution会有所有的步骤

string 和 string1 最简单的步骤是前者加个1，这个操作我们就当作把第二个字符串去掉最后一个

**add**: 字符串1 添加一个 字符串2 的tail  字符串2222222用左移
<div> **del**：字符串1 删掉一个 字符，   字符串11111用左移 </div>
<div>**sub**：字符串替换操作，如果相同直接两个都左移， 如果不相同先替换后两个左移, 这也等同于先删除字符串1的tail又增加一个跟字符串2tail一样的到字符串1 ，直观一点， 把sub的+2变成+1

In [11]:
@lru_cache(maxsize = 2**10)
def edit_simple(string1, string2):
    if len(string1) == 0: return len(string2)
    if len(string2) == 0: return len(string1)
    
    tail_s1 = string1[-1]
    tail_s2 = string2[-1]
    return min([
        edit_distance(string1[:-1], string2)+1,
        edit_distance(string1,string2[:-1])+1, # add操作，删第二个字符串的字符
        edit_distance(string1[:-1], string2[:-1]) + (0 if tail_s1 == tail_s2 else 2),
    ])

In [12]:
edit_simple('1010','11100')

2

In [13]:
solution = {}
@lru_cache(maxsize=2**10)
def edit_distance(string1, string2):
    
    if len(string1) == 0: return len(string2)
    if len(string2) == 0: return len(string1)
    
    tail_s1 = string1[-1]
    tail_s2 = string2[-1]
    
    candidates = [
        (edit_distance(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)),  # string 1 delete tail
        (edit_distance(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2)),  # string 1 add tail of string2
    ]
    
    if tail_s1 == tail_s2:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 0, '')
    else:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 1, 'SUB {} => {}'.format(tail_s1, tail_s2))

    candidates.append(both_forward)
    
    min_distance, operation = min(candidates, key=lambda x: x[0])
    
    solution[(string1, string2)] = operation 
    
    return min_distance

In [3]:
edit_distance('今天我瘦了','今天我胖了')

1

In [4]:
edit_distance('AACDE','ABC')

3

In [6]:
solution = {}

In [7]:
edit_distance('1010','11100')

2

In [8]:
solution

{('1', '1'): '',
 ('1', '11'): 'ADD 1',
 ('1', '111'): 'ADD 1',
 ('1', '1110'): 'ADD 0',
 ('1', '11100'): 'ADD 0',
 ('10', '1'): 'DEL 0',
 ('10', '11'): 'SUB 0 => 1',
 ('10', '111'): 'ADD 1',
 ('10', '1110'): '',
 ('10', '11100'): 'ADD 0',
 ('101', '1'): 'DEL 1',
 ('101', '11'): '',
 ('101', '111'): '',
 ('101', '1110'): 'ADD 0',
 ('101', '11100'): 'ADD 0',
 ('1010', '1'): 'DEL 0',
 ('1010', '11'): 'DEL 0',
 ('1010', '111'): 'DEL 0',
 ('1010', '1110'): '',
 ('1010', '11100'): 'ADD 0'}

## Edit-distance-solution(两个DNA的相似度，变化越大越不接近)
    采用 分治的思想，将复杂问题分解成相似的子问题，然后求解各个子问题
    假设字符串A[m]，转换到字符串B[n]
    从A的最后一位开始，通过一系列操作转变成B
    
   > 该问题具有如下的递归规律：
    > 1. A[m] == B[n]，从A最后一位转变成B的最后一位，没有操作，
    > 2. A[m]!=B[n]， 有以下几种情况：
   * 删除最后一位，string1 和 string， 前者减1
   * 增加最后一位，string 和 string1，前者加1
   * 替换最后一位，string 和 strin1 , 把g换成1
    >> 递归边界：当观察的字符串长度length == 0， 递归跳出
   
   

In [9]:
op = []
def parse_solution(left, right): # 得到每一步的步骤
    if len(left) == 0: return len(right)
    if len(right) == 0: return len(left)

    temp = solution[(left, right)]
    if temp == '': return parse_solution(left[:-1], right[:-1])
    if 'DEL' in temp: 
        op.append(temp)
        return parse_solution(left[:-1],right)
    
    if 'ADD' in temp:
        op.append(temp)
        return parse_solution(left, right[:-1])
    if 'SUB' in temp:
        op.append(temp)
        return parse_solution(left[:-1], right[:-1])
    return 

In [10]:
parse_solution('1010','11100')
op 

['ADD 0', 'SUB 0 => 1']